# 3.1 Elementary Polygraphic Substitution Ciphers

## Exercises 3.1

In [1]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().absolute().parent))

from src.helpers import strip_text, format_ciphertext
from src.polygraphic_ciphers import validate_playfair_key, format_digraph_plaintext


### 3. Write a computer program to implement Playfair's method. The program should be capable of both enciphering and deciphering messages using this system.

In [2]:
OMITTED_LETTER = 'J'
PADDING_LETTER = 'Q'
DOUBLE_LETTER_REPLACEMENT = 'X'
class PlayfairCipher:
    """
    Implementation of Playfair's cipher
    """
    def __init__(self, key: list[list[str]]):
        validate_playfair_key(key, OMITTED_LETTER, PADDING_LETTER, DOUBLE_LETTER_REPLACEMENT)
        self._key = key
    @staticmethod
    def _mod_5(i: int) -> int:
        # get rid of negative indexes
        while i < 0:
            i += 5
        # handle wrap around
        return i % 5
    def _find_position(self, char: str) -> tuple[int, int] | None:
        for i, row in enumerate(self._key):
            for j, c in enumerate(row):
                if c == char:
                    return i, j
        return None
    def _char_at_position(self, pos: tuple[int, int]) -> str:
        return self._key[PlayfairCipher._mod_5(pos[0])][PlayfairCipher._mod_5(pos[1])]
    def encipher(self, plaintext: str) -> str:
        """
        Encipher a plaintext message using Playfair's method.
        """
        plaintext = strip_text(plaintext).upper()
        # remove the omitted letter
        plaintext = plaintext.replace(OMITTED_LETTER, '')
        #  pad plaintext if the length is odd
        if len(plaintext) % 2 != 0:
            plaintext += PADDING_LETTER
        ciphertext = []
        # examine the pairs in the plaintext and apply the cipher rules
        for i in range(0, len(plaintext), 2):
            pos_1 = self._find_position(plaintext[i])
            # replace the second of two consecutive letters with the double letter replacement
            second_letter = plaintext[i+1] if plaintext[i+1] != plaintext[i] else DOUBLE_LETTER_REPLACEMENT
            pos_2 = self._find_position(second_letter)
            if pos_1 is None or pos_2 is None:
                raise ValueError("Could not find position for " +
                                 f"{plaintext[i]} or {second_letter}")
            if pos_1[0] != pos_2[0] and pos_1[1] != pos_2[1]:
                # rectangle corners, add the letters at the opposite corners
                ciphertext.append(self._char_at_position((pos_1[0], pos_2[1])))
                ciphertext.append(self._char_at_position((pos_2[0], pos_1[1])))
            elif pos_1[0] == pos_2[0]:
                # same row, add 1 to the column for each letter
                ciphertext.append(self._char_at_position((pos_1[0], pos_1[1]+1)))
                ciphertext.append( self._char_at_position((pos_2[0], pos_2[1]+1)))
            else:
                # same column, add 1 to the row for each letter
                ciphertext.append(self._char_at_position((pos_1[0]+1, pos_1[1])))
                ciphertext.append(self._char_at_position((pos_2[0]+1, pos_2[1])))
        return ''.join(ciphertext)
    def decipher(self, ciphertext: str) -> str:
        """"
        Decipher a ciphertext message using Playfair's method.
        """
        ciphertext = strip_text(ciphertext).upper()
        if len(ciphertext) % 2 != 0:
            raise ValueError("Ciphertext must be in pairs of letters")
        plaintext = []
        # examine the pairs in the ciphertext and apply the decipher rules
        for i in range(0, len(ciphertext), 2):
            pos_1 = self._find_position(ciphertext[i])
            pos_2 = self._find_position(ciphertext[i+1])
            if pos_1 is None or pos_2 is None:
                raise ValueError("Could not find position for " +
                                 f"{ciphertext[i]} or {ciphertext[i+1]}")
            if pos_1[0] != pos_2[0] and pos_1[1] != pos_2[1]:
                # rectangle corners, add the letters at the opposite corners
                plaintext.append(self._char_at_position((pos_1[0], pos_2[1])))
                plaintext.append(self._char_at_position((pos_2[0], pos_1[1])))
            elif pos_1[0] == pos_2[0]:
                # same row, subtract 1 to the column for each letter
                plaintext.append(self._char_at_position((pos_1[0], pos_1[1]-1)))
                plaintext.append(self._char_at_position((pos_2[0], pos_2[1]-1)))
            else:
                # same column, subtract 1 to the row for each letter
                plaintext.append(self._char_at_position((pos_1[0]-1, pos_1[1])))
                plaintext.append(self._char_at_position((pos_2[0]-1, pos_2[1])))
        return ''.join(plaintext).lower()


### 1. Encipher the following messages using Playfair's system and the arrangement of letters appearing in Example 3.2:

In [3]:
EXAMPLE_3_2_KEY = [
    ['D', 'B', 'M', 'W', 'I'],
    ['C', 'O', 'X', 'G', 'E'],
    ['Q', 'Y', 'R', 'F', 'S'],
    ['Z', 'A', 'K', 'T', 'P'],
    ['L', 'U', 'H', 'N', 'V'],
]
cipher = PlayfairCipher(EXAMPLE_3_2_KEY)

#### (a) Even though the Cold War has come and gone, its chill winds still blow.

In [4]:
plaintext_1a = "Even though the Cold War has come and gone, its chill winds still blow."
ciphertext_1a = cipher.encipher(plaintext_1a)
print(format_ciphertext(ciphertext_1a))

SIGVK NYBXN KNCOC UBIKY UKQEX BOPLW EXVGW PQEVM HCIDL WREPW
HCDUG B


#### (b) All those who believe in psychokinesis raise my hand.

In [5]:
plaintext_1b= "All those who believe in psychokinesis raise my hand."
ciphertext_1b = cipher.encipher(plaintext_1b)
print(format_ciphertext(ciphertext_1b))

ZUNZU XPSMN YOCVE SISWV VPQOU XPMVG PEQFP BPSBR UKLW



### 2. Decipher the following messages that were enciphered using Playfair's system and the arrangement of letters appearing in Example 3.2:
#### (a) GVXSE WPCYM HVUFK YSXFP CIXLK YECBI WPUMK PPGFR.

In [6]:
ciphertext_2a = "GVXSE WPCYM HVUFK YSXFP CIXLK YECBI WPUMK PPGFR"
plaintext_2a = cipher.decipher(ciphertext_2a)
print(format_digraph_plaintext(plaintext_2a))

en er gi ze rb un ny ar re st ed ch ar ge dw it hb at te ry


yields
```
  energizer bunny arrested charged with battery
```


#### (b) EICEG AKZUX AGXFK ZVMXD XIXYS BVYZW GUKNP UOPUF SWHD.



In [7]:
ciphertext_2b = "EICEG AKZUX AGXFK ZVMXD XIXYS BVYZW GUKNP UOPUF SWHD"
plaintext_2b = cipher.decipher(ciphertext_2b)
print(format_digraph_plaintext(plaintext_2b))

iv eg ot ap ho to gr ap hi cm em or yi us td on th av ea ny
fi lm


yields
```
  i've got a photographic memory i just don't have any
  film
```